# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
#импорт используемых библиотек
import pandas as pd
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer

In [2]:
df = pd.read_csv('/datasets/data.csv') #чтение файла csv в датафрейм df

In [3]:
df.info() #вывод общей информации о датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.head(5) #вывод первых 5 строк датафрейма

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

В файле 12 колонок, 2 колонки типа float, 5 колонок типа int и 5 колонок типа object.

## Шаг 2. Предобработка данных

### Обработка пропусков

#### Столбец children

In [5]:
df['children'].value_counts() #получение уникальных значений

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце children есть неясные значения (-1 и 20), но пропусков нет  
Можно предположить, что это ошибки, возникшие при заполнении анкеты, и заменить значение -1 на 1, а 20 заменить на 2

In [6]:
df.loc[(df['children'] == -1), 'children'] = 1 #замена в столбце children значений, равных -1

df.loc[(df['children'] == 20), 'children'] = 2#замена в столбце children значений, равных 20

df['children'].value_counts() #получение уникальных значений

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Теперь данные похожи на правду!

#### Столбец days_employed

In [7]:
df['days_employed'].value_counts() #получение уникальных значений

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64

Так как в столбце слишком много различных значений, то значений NaN (если они есть) не видно. Чтобы проверить их наличие, воспользуемся методом isna()

In [8]:
len(df[df['days_employed'].isna()]) #вывод количества тех строк, где есть NaN

2174

Ого, в этом столбце целых 2147 пропусков!  

Заменим пропущенные значения на среднее при помощи метода mean()

In [9]:
df['days_employed'] = df['days_employed'].fillna(value = df['days_employed'].mean()) 
#замена пропущенных значений средним по столбцу

len(df[df['days_employed'].isna()])  #вывод количества тех строк, где в заданном столбце есть NaN

0

Пропуски устранены

#### Столбец dob_years

In [10]:
#df['dob_years'].value_counts().sort_index() #получение уникальных значений и сортировка для удобства восприятия
df['dob_years'].describe() #лучше использовать describe(), более компактный вывод и сразу видно min и max

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

В этом столбце пропуском можно считать значение 0  
Заменим его на средний возраст при помощи метода mean()

In [11]:
df.loc[(df['dob_years'] == 0), 'dob_years'] = int(df['dob_years'].mean()) 
#замена значения 0 средним по столбцу и приведение к типу int, т.к среднее - вещественное
df['dob_years'].describe()

count    21525.000000
mean        43.495145
std         12.218213
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Теперь видно, что минимальный возраст 19, а максимальный - 75  
Подходит!

#### Столбец education

In [12]:
#df['education'].value_counts() #получение уникальных значений
df['education'].unique() #лучше использовать unique() для компактности вывода

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Пропусков нет

#### Столбец education_id

In [13]:
df['education_id'].value_counts() #получение уникальных значений

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Тут все отлично

#### Столбец family_status

In [14]:
df['family_status'].value_counts() #получение уникальных значений

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Все хорошо

#### Столбец family_status_id

In [15]:
df['family_status_id'].value_counts() #получение уникальных значений

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Супер

#### Столбец gender

In [16]:
df['gender'].value_counts() #получение уникальных значений

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Один раз встречается пропуск  
Заменим его на F, т.к этот пол встречается почти вдвое чаще

In [17]:
df.loc[(df['gender'] == 'XNA'), 'gender'] = 'F' #замена в столбце gender значения, равного XNA
df['gender'].value_counts() #получение уникальных значений

F    14237
M     7288
Name: gender, dtype: int64

#### Столбец income_type

In [18]:
df['income_type'].value_counts() #получение уникальных значений

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Отлично

#### Столбец dept

In [19]:
df['debt'].value_counts() #получение уникальных значений

0    19784
1     1741
Name: debt, dtype: int64

Супер

#### Столбец total_income

In [20]:
df['total_income'].value_counts() #получение уникальных значений

169846.427535    1
257737.077768    1
200508.675866    1
106196.235958    1
248730.171354    1
                ..
175057.266090    1
101516.604975    1
239154.168013    1
165009.733021    1
189255.286637    1
Name: total_income, Length: 19351, dtype: int64

Снова воспользуемся методом isna()

In [21]:
len(df[df['total_income'].isna()]) #вывод количествоа тех строк, где в заданном столбце есть NaN

2174

Тоже 2174 пропусков, как в столбце days_employed  
Тут явно есть связь...  
Заменим на медианное значение

In [22]:
df['total_income'] = df['total_income'].fillna(value=df['total_income'].median())
#замена пропущенных значенией медианным по столбцу

len(df[df['total_income'].isna()]) #вывод количества тех строк, где в заданном столбце есть NaN

0

#### Столбец purpose

In [23]:
df['purpose'].value_counts() #получение уникальных значений

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Пропусков нет

### Замена типа данных

У столбцов days_employed и total_income данные представлены в вещественном типе, хотя никакой роли дробная часть не играет  
Представим эти столбцы в целочисленном типе с помощью метода astype(), т.к все пропуски, могущие вызвать ошибку, устранены в предыдущем разделе

#### Столбец days_employed

In [24]:
df['days_employed'] = df['days_employed'].astype('int') #приведение к целочисленному типу
df.head(5) #вывод первых пяти строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Данные в столбце days_employed какие-то странные, где-то значения отрицательные, а где-то при переводе в года дают нереальные числа  
Хотелось бы уточнить у разработчиков, как получились такие данные и получить корректные значения  
Видимо, использовать этот столбец при анализе не получится

#### Столбец total_income

In [25]:
df['total_income'] = df['total_income'].astype(int) #приведение к целочисленному типу
df.head(5) #вывод первых пяти строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Теперь числа в столбце удобнее для восприятия

#### Столбец debt

In [26]:
df['debt'] = df['debt'].astype(bool) #приведение к целочисленному типу
df.head(5) #вывод первых пяти строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу


Debt действительно лучше переводить в bool

### Обработка дубликатов

#### Столбец education

In [27]:
df['education'].value_counts() #получение уникальных значений

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Приведем все значения к нижнему регистру методом str.lower()

In [28]:
df['education']=df['education'].str.lower() #приведение к нижнему регистру

df['education'].value_counts() #получение уникальных значений

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [29]:
df['education'].duplicated()

0        False
1        False
2         True
3         True
4         True
         ...  
21520     True
21521     True
21522     True
21523     True
21524     True
Name: education, Length: 21525, dtype: bool

In [30]:
df['education'].duplicated().sum()

21520

In [31]:
df.duplicated().sum()

71

In [32]:
df = df.drop_duplicates().reset_index(drop=True) 

In [33]:
df.duplicated().sum()

0

### Лемматизация

In [34]:
m = Mystem() #создание объекта класса Mystem

In [35]:
df['purpose'].value_counts() #получение уникальных значений

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Произведем лемматизацию

In [36]:
df['lemmas'] = df['purpose'].apply(m.lemmatize) 
#лемматизация значений в столбце purpose и запись их в столбец lemmas

In [37]:
df['lemmas'].value_counts().to_frame() #получение уникальных значений

,lemmas
"[автомобиль, \n]",972
"[свадьба, \n]",791
"[на, , проведение, , свадьба, \n]",768
"[сыграть, , свадьба, \n]",765
"[операция, , с, , недвижимость, \n]",675
"[покупка, , коммерческий, , недвижимость, \n]",661
"[операция, , с, , жилье, \n]",652
"[покупка, , жилье, , для, , сдача, \n]",651
"[операция, , с, , коммерческий, , недвижимость, \n]",650
"[покупка, , жилье, \n]",646


Много дублирующихся значений

In [38]:
lemmas = df['lemmas'].value_counts().index.tolist() 
#скопируем из столбца lemmas уникальные значения (списки слов), т.е индексы, в список lemmas

#получился список списков, преобразуем в единый список
lemmas_list = [] #пустой список, куда добавим вложенные списки

for lemma in lemmas: #пройдемся по всему списку
    lemmas_list.extend(lemma) #добавим список в новый методом extend()
    
words = sorted(list(set(lemmas_list))) 
#получим сортированный список, при этом для уникальности значений используем преобразование в set

words

['\n',
 ' ',
 'автомобиль',
 'высокий',
 'для',
 'дополнительный',
 'жилой',
 'жилье',
 'заниматься',
 'коммерческий',
 'на',
 'недвижимость',
 'образование',
 'операция',
 'подержанный',
 'подержать',
 'покупка',
 'получение',
 'приобретение',
 'проведение',
 'профильный',
 'ремонт',
 'с',
 'свадьба',
 'свой',
 'сдача',
 'сделка',
 'семья',
 'со',
 'собственный',
 'строительство',
 'сыграть']

С данными в таком виде гораздо проще и удобнее работать  
Выделим из этого списка ключевые слова и произведем стемминг для получения основ слов

In [39]:
russian_stemmer = SnowballStemmer('russian')

In [40]:
words = ['автомобиль', 'жилье', 'недвижимость', 'образование', 'свадьба'] #список ключевых слов

for word in words:
    print ('Исходное слово: ' + word + ', после стемминга: ' + russian_stemmer.stem(word)) #стемминг слов

Исходное слово: автомобиль, после стемминга: автомобил
Исходное слово: жилье, после стемминга: жил
Исходное слово: недвижимость, после стемминга: недвижим
Исходное слово: образование, после стемминга: образован
Исходное слово: свадьба, после стемминга: свадьб


### Категоризация данных

#### Категоризация по цели кредита

В предыдущем разделе мы обработали текстовые данные, таким образом получив список основ слов, при помощи которых можно провести категоризацию по цели кредита  

Всего можно выделить 4 цели:
* Автомобиль
* Недвижимость
* Образование
* Свадьба

In [41]:
def category(purpose): #функция, возвращающая значение категории
    if 'автомобил' in purpose:
        return 'Автомобиль'
    if 'жил' in purpose or 'недвижим' in purpose:
        return 'Недвижимость'
    if 'образован' in purpose:
        return 'Образование'
    if 'свадьб' in purpose:
        return 'Свадьба'

In [42]:
df['purpose_category'] = df['purpose'].apply(category) 
#получение столбца purpose_category, содержащего категорию цели кредита

In [43]:
df['purpose_category'].value_counts() #вывод количества уникальных значений

Недвижимость    10811
Автомобиль       4306
Образование      4013
Свадьба          2324
Name: purpose_category, dtype: int64

Получили столбец датафрейма, содержащий нужную категорию для каждой цели кредита

#### Категоризация по наличию или отсуствию детей

Произведем категоризацию по наличию или отсутствию детей  
Так как категорий всего две, разумно использовать тип bool

In [44]:
def category(children): #функция, возвращающая значение категории
    if children == 0:
        return False #если детей нет, вернуть False
    else:
        return True #если дети есть, вернуть True

In [45]:
df['children_category'] = df['children'].apply(category) 
#получение столбца children_category, содержащего категорию наличия детей

In [46]:
df['children_category'].value_counts() #вывод количества уникальных значений

False    14091
True      7363
Name: children_category, dtype: int64

Получили столбец датафрейма, содержащий нужную категорию для наличия или отсутствия детей

#### Категоризация по уровню дохода

Произведем категоризацию по уровню дохода  
Воспользуемся методом describe() для вычисления сводных статистик

In [47]:
df['total_income'].describe().astype('int')

count      21454
mean      165225
std        98021
min        20667
25%       107623
50%       145017
75%       195813
max      2265604
Name: total_income, dtype: int64

In [48]:
df['quantile_id'] = pd.qcut(df['total_income'], q=4,labels=False)
df['quantile_id'].value_counts()

1    6415
0    5364
3    5364
2    4311
Name: quantile_id, dtype: int64

In [49]:
df['quantile'] = pd.qcut(df['total_income'], q=4)
df['quantile'].value_counts()

(107623.0, 145017.0]      6415
(20666.999, 107623.0]     5364
(195813.25, 2265604.0]    5364
(145017.0, 195813.25]     4311
Name: quantile, dtype: int64

## Шаг 3. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

В предыдущем пункте мы получили категории для наличия или отсутствия детей  
Теперь необходимо вывести взаимосвязь между наличием детей и возвратом кредита в срок

Для начала сформируем датафрейм, который будет содержать сводные данные по категории и количеству "хороших" и "плохих" задолженностей

Напишем функцию, которая будет возвращать Series из количества сгруппированных и отфильтрованных по debt записей

In [50]:
def df_category_debt(column, debt):
    return df[df['debt'] == debt].groupby(column)['debt'].count()

In [51]:
df_children = df_category_debt('children_category', True).to_frame()

df_children['ok'] = df_category_debt('children_category', False)
#проделаем то же самое для записей, где просрочка отсуствует, 
#после чего получившийся столбец добавим к датафрейму

df_children #выведем датафрейм на экран

,debt,ok
children_category,,
False,1063,13028
True,678,6685


Получили количество просрочек и выплаченных задолженностей для каждой категории  
Но полученные абсолютные значения неинформативны, следовательно, нужно вычислить относительные значения (проценты)

Напишем функцию, которая будет возвращать Series из процентов просроченных задолженностей

In [52]:
def percent (df, col1, col2):
    return 100 * df[col1] / (df[col2] + df[col1])

In [53]:
df_children['%'] = percent(df_children,'debt','ok')
df_children.sort_values('%')

,debt,ok,%
children_category,,,
False,1063,13028,7.543822
True,678,6685,9.208203


Таким образом, выяснили, что имеющие детей допускают просрочки в 10% случаев  
При этом не имеющие детей допускают просрочки в 8% случаев  
Следовательно, наличие детей немного увеличивает вероятность просрочки

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Исходные данные категоризированы по семейному положению, следовательно, необходимо провести аналогичные действия для этих категорий

Для начала сформируем датафрейм, который будет содержать сводные данные по категории и количеству "хороших" и "плохих" задолженностей
Воспользуемся уже написанными функциями df_category_debt() и percent()

In [54]:
df_family_status = df_category_debt('family_status', True).to_frame()

df_family_status['ok'] = df_category_debt('family_status', False)

df_family_status

,debt,ok
family_status,,
Не женат / не замужем,274,2536
в разводе,85,1110
вдовец / вдова,63,896
гражданский брак,388,3763
женат / замужем,931,11408


In [55]:
df_family_status['%'] = percent(df_family_status,'debt','ok')
df_family_status.sort_values('%')

,debt,ok,%
family_status,,,
вдовец / вдова,63,896,6.569343
в разводе,85,1110,7.112971
женат / замужем,931,11408,7.545182
гражданский брак,388,3763,9.347145
Не женат / не замужем,274,2536,9.750890


Получились довольно интересные результаты  
Самые дисциплинированные - вдовцы и разведенные  
Состоящие в браке - крепкие середнячки  
А большее число просрочек у состоящих в "гражданском браке" и несемейных

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

В предыдущем пункте мы получили категории для распределения доходов, разделив данные на 4 группы по квантилям
Теперь необходимо вывести взаимосвязь между уровнем дохода и просрочками

Для начала сформируем датафрейм, который будет содержать сводные данные по категории и количеству "хороших" и "плохих" задолженностей
Воспользуемся уже написанными функциями df_category_debt() и percent()

In [56]:
df_quantile = df_category_debt('quantile_id', True).to_frame()

df_quantile['ok'] = df_category_debt('quantile_id', False)

df_quantile

,debt,ok
quantile_id,,
0,427,4937
1,547,5868
2,384,3927
3,383,4981


In [57]:
df_quantile['%'] = percent(df_quantile,'debt','ok')
df_quantile.sort_values('%')

,debt,ok,%
quantile_id,,,
3,383,4981,7.140194
0,427,4937,7.960477
1,547,5868,8.526890
2,384,3927,8.907446


Получается, что лучше всех выплачивают кредиты самые обеспеченные, потом самые бедные, а у середнячков число просрочек растет

### Как разные цели кредита влияют на его возврат в срок?

В предыдущем пункте мы провели лемматизацию и стемминг, получив 4 категории для целей получения кредита
Теперь необходимо вывести взаимосвязь между целью кредита и просрочками

Для начала сформируем датафрейм, который будет содержать сводные данные по категории и количеству "хороших" и "плохих" задолженностей
Воспользуемся уже написанными функциями df_category_debt() и percent()

In [58]:
df_purpose_category = df_category_debt('purpose_category', True).to_frame()

df_purpose_category['ok'] = df_category_debt('purpose_category', False)

df_purpose_category

,debt,ok
purpose_category,,
Автомобиль,403,3903
Недвижимость,782,10029
Образование,370,3643
Свадьба,186,2138


In [59]:
df_purpose_category['%'] = percent(df_purpose_category,'debt','ok')
df_purpose_category.sort_values('%')

,debt,ok,%
purpose_category,,,
Недвижимость,782,10029,7.233373
Свадьба,186,2138,8.003442
Образование,370,3643,9.220035
Автомобиль,403,3903,9.359034


Выходит так, что ипотечники платят стабильнее всех, за ними идут молодожены, а на кредиты на образование и автомобиль чаще "забивают"

## Шаг 4. Общий вывод

Были проанализированы зависимости различных категорий заемщиков и количества просрочек  
Но насколько полученные данные значимы?

Сформируем датафрейм, в котором соберем сводные статистики по всем изученным зависимостям

In [60]:
def df_final(column, series):
    return series.describe()
#функция, формирующая столбец из сводной статистики

In [61]:
columns = ['children', 'family', 'quantile', 'purpose'] #список колонок

series = [df_children['%'], df_family_status['%'], df_quantile['%'], df_purpose_category['%']]
#список Series, из которых извлекаем статистику

In [62]:
final = pd.DataFrame() #объявление итогового датафрейма со статистикой

for i in range(len(series)):
    final[columns[i]] = df_final(columns[i], series[i])
#в цикле формируем столбцы датафрейма
    
final #выведем датафрейм на экран

,children,family,quantile,purpose
count,2.000000,5.000000,4.000000,4.000000
mean,8.376013,8.065106,8.133752,8.453971
std,1.176895,1.405319,0.768189,1.016338
min,7.543822,6.569343,7.140194,7.233373
25%,7.959918,7.112971,7.755406,7.810925
50%,8.376013,7.545182,8.243684,8.611739
75%,8.792108,9.347145,8.622029,9.254785
max,9.208203,9.750890,8.907446,9.359034


Получается, что корреляция есть, но она не очень значительная  
В любом случае, число просрочек находится в диапазоне 7-10%, в среднем 9%  
Нужно использовать еще какие-то данные